In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.manifold import MDS
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import skfuzzy as fuzz

# Preprocess:
read csv and normalization

In [76]:
data = pd.read_csv("e-shop data and description/e-shop clothing 2008.csv", delimiter=";")
data = data.drop(['year', 'page 2 (clothing model)'], axis=1)
scaler = StandardScaler()
data = pd.DataFrame(data=scaler.fit_transform(data), columns=[data.columns])

# Clustering:

### k means

In [ ]:
km_n_clusters = 6
kmeans = KMeans(n_clusters=km_n_clusters, algorithm='full')
kmeans_labels = kmeans.fit_predict(data)

### gmm

In [ ]:
gmm_n_clusters = 6
gmm = GaussianMixture(n_components=gmm_n_clusters, covariance_type='full', init_params='kmeans'
                          , warm_start=False, n_init=5, random_state=100, verbose=2)
gmm.fit(data)
gmm_labels = gmm.predict(data)

### fuzzy c means

In [88]:
fuzzy_n_clusters = 6
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(data.T.values, fuzzy_n_clusters, 2, error=0.005, maxiter=1000)
fuzzy_labels = np.argmax(u, axis=0)

# Dimension Reduction:

In [87]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(data)
print(pca.explained_variance_ratio_)
print("\n total variance:", str(int((sum(pca.explained_variance_ratio_)*100))) + "%", "\n")
pca_data = pd.DataFrame(data=principal_components, columns = ['dim 1', 'dim 2'])

[0.1731577  0.16582818]

 total variance: 33% 



### plot clusters:

In [85]:
%matplotlib qt5

In [82]:
def cluster_color(label):
    colors = ['b', 'g', 'r', 'c', 'm', '#fdff03', '#055803', '#a6fc00', '#9d70d1', '#ff703b', '#3a70d8', '#ff70b5']
    return colors[label % len(colors)]

In [83]:
def plot_clusters(data, labels, n_clusters, lfs=10, tfs=8):
    for label in range(n_clusters):
        plt.scatter(data.values[labels==label, 0], data.values[labels==label, 1], s=40, c=cluster_color(label))
    clusters_ax = plt.gca()
    clusters_ax.set_xlabel('dim1', fontsize=lfs)
    clusters_ax.set_ylabel('dim2', fontsize=lfs)
    plt.show()

In [89]:
plot_clusters(pca_data, fuzzy_labels, fuzzy_n_clusters)